In [ ]:
from Bio import SeqIO
import os
import pandas as pd
import itertools
import math
import numpy as np
from scipy.stats import binom
from statsmodels.stats import multitest

**Author:** Jeremy Owen


**Description:** A preliminary attempt at analysing patterns of regulatory gene co-occurance in biosynthetic gene clusters.


**Link for dataset:** https://zenodo.org/record/7658989#.Y_Pkh-xBxJU

In [ ]:
#download dataset from link in markdown cell above, add link to unzipped folder below
big_strep_dir = "/put/your/path/here"

In [1]:
def count_cds_features_with_pattern(genbank_file, pattern):
    # Read the GenBank file
    record = SeqIO.read(genbank_file, "genbank")

    # Iterate over all the features in the GenBank record
    counts = {}
    for feature in record.features:
        # Check if the feature is a CDS feature and contains the pattern
        if feature.type == "CDS" and "gene_kind" in feature.qualifiers:
            if pattern in feature.qualifiers["gene_kind"]:
                #print (feature.qualifiers["gene_functions"])
                try:
                    reg_type = feature.qualifiers["gene_functions"][0].split(":")[1].split("(")[0]
                    
                except:
                    #strange formatting crops up sometimes, try/except to prevent breakage
                    #print(feature.qualifiers["gene_functions"][0], feature.qualifiers["gene_kind"])
                    reg_type = feature.qualifiers["gene_functions"][0]
                
                if "E-value" in reg_type:
                    #print(feature.qualifiers["gene_functions"][0], feature.qualifiers["gene_kind"])
                    #reg_type = reg_type = feature.qualifiers["gene_functions"][0]
                    reg_type = "pkinase"
                    # in every case examined of like a thousand, this was the case
                #print(reg_type)
                counts[reg_type] = counts.get(reg_type, 0) + 1

    # Return the counts as a dictionary
    return(counts)

In [8]:
def get_gbk_files(input_dir,exlude_pattern = 'genome'):
    gbk_files = []
    for filename in os.listdir(input_dir):
        if filename.endswith(".gbk") and not exlude_pattern in filename:
            filepath = os.path.join(input_dir, filename)
            gbk_files.append(filepath)
    return gbk_files


In [10]:
def parse_gbk_directory(input_dir):
    all_counts={}
    gbk_files = get_gbk_files(input_dir)
    for f in gbk_files:
        #key=f.split("/")[-1].split(".")[0]
        counts = count_cds_features_with_pattern(f,'regulatory')
        if counts:
            all_counts[f] = counts
    return all_counts

In [217]:
big_strep_counts = parse_gbk_directory(big_strep_dir)

In [202]:
#Should return True
len(big_strep_counts) == 8712

True

In [148]:
#get rid of the massive index names, but preserve mapping (it might come in handy)
file_map = {}
strep_dict = {}
i = 1
for key in big_strep_counts:
    new_key = 'bgc' + str(i)
    file_map[new_key] = key
    strep_dict[new_key] = big_strep_counts[key].copy()
    i+=1

In [149]:
sums = {} 
for key1 in big_strep_counts:
    for key2 in big_strep_counts[key1]:
        #print(key2)
        sums[key2] = sums.get(key2,0) + 1

In [150]:
def generate_dataframe(obs_dict):
    """
    makes a data frame from the dictionary generated by parse_gbk_directory
    """
    # Initialize an empty dictionary of feature counts
    all_features = set()
    counts = {}
    for obs_id, obs_counts in obs_dict.items():
        # Add the feature counts to the dictionary
        for feature, count in obs_counts.items():
            all_features.add(feature)
            if obs_id not in counts:
                counts[obs_id] = {}
            counts[obs_id][feature] = count

    # Convert the dictionary to a DataFrame and fill missing values with zeros
    df = pd.DataFrame.from_dict(counts, orient='index')
    df = df.fillna(0)

    # Reorder the columns in alphabetical order
    df = df[sorted(all_features)]
    
    return df

In [151]:
strep_df = generate_dataframe(strep_dict)

In [203]:
#should return True
len(strep_df.columns)==30

True

In [153]:
def count_to_binary(df, threshold=0):
    """
    converts dataframe from counts, to binary presence/absense
    done to simplify statistical tests downstream
    """
    # Create a copy of the input DataFrame
    binary_df = df.copy()

    # Replace all values above the threshold with 1 and all others with 0
    binary_df[binary_df <= threshold] = 0
    binary_df[binary_df > threshold] = 1
    
    return binary_df

In [154]:
binary_strep_df = count_to_binary(strep_df)

In [155]:
to_drop = ['LuxR family DNA-binding response regulator ',
 'LuxR family transcriptional regulator ',
 'transcriptional regulator ',
 'response regulator ',
 'pkinase']

In [156]:
final_df = binary_strep_df.drop(to_drop, axis=1)
# remove non regulators, generics, and TCS response regulators. TCS now counted by SensorHK only

In [201]:
#should return True
len(final_df.columns)==25


True

In [158]:
def calc_binom(n, k):
    return math.comb(n, k)

def count_feature_combinations(df, k, n=None):
    """
    Generates all k-length comninations for features in a df
    counts the number of times these occur in the df
    reports the combinations with the n highest counts
    """
    # Initialize a dictionary to store the feature combinations and their counts
    feature_counts = {}
    #if the number of combinations to report is not specified, set it to all possible combinations
    if n is None:
        n = calc_binom(len(df.columns),k)
        

    # Iterate over all combinations of length k
    for combination in itertools.combinations(df.columns, k):
        # Count the number of observations that have all of the features in the combination
        observations_with_features = df[list(combination)].min(axis=1).sum()

        # Add the feature combination and its count to the dictionary
        feature_counts[combination] = observations_with_features

    # Sort the dictionary by count and return the top n entries
    top_n_combinations = dict(sorted(feature_counts.items(), key=lambda x: x[1], reverse=True)[:n])

    return top_n_combinations

In [159]:
combs_4 = count_feature_combinations(final_df, 4)

In [204]:
#should return True
len(combs_4) == 12650

True

In [161]:
def calculate_feature_set_probability(df, feature_set):
    """
    Calculate the probability of a feature set occurring in a DataFrame by assuming independence.
    
    Parameters:
    df (pandas DataFrame): The input DataFrame of binary observations.
    feature_set (list): A list of feature names to calculate the probability for.
    
    Returns:
    The probability of the feature set occurring in the DataFrame by assuming independence.
    This is the product of the probabilities of each feature
    
    !!Only works for binary dataframes!!
    """
    # calculate the frequency of each feature
    feature_frequencies = df[feature_set].mean()
    
    # calculate the probability of the feature set occurring by assuming independence
    feature_set_probability = np.prod(feature_frequencies)
    
    return feature_set_probability

In [162]:
def calculate_cumulative_binomial_probability(n, p, num_trials):
    """
    Calculate the probability of n or more successes using the cumulative binomial distribution.
    
    Parameters:
    n (int): The minimum number of successes to consider.
    p (float): The probability of success in each trial.
    num_trials (int): The total number of trials.
    
    Returns:
    The probability of n or more successes using the cumulative binomial distribution.
    """
    # calculate the probability of n-1 or fewer successes
    probability = binom.cdf(n-1, num_trials, p)
    
    # subtract this probability from 1 to get the probability of n or more successes
    probability = 1 - probability
    
    return probability


In [163]:
calculate_cumulative_binomial_probability(52,0.002897080319329784,11000)
#work this out with a pencil if you don't trust me

0.0006360278376437156

In [164]:
def report_pvalues(combs, df, n=11975,cutoff=0.001):
    """
    Takes a dictionary of combination frequencies, and the dataframe from which they were derived
    Calculates probability assuming indepndence
    Uases observed count, binom distribution, and number of rows(trials) to calc p for oberved
    Records and reporets if below cutoff
    !! note, n is set to 11975 in this case, as this was the number of BGCs in the originnal input
    !! Some of them had no regulatory elements, so the df dimension is smaller, like 8k or something
    !! Doesn't correct for multiple comparisons. Fixed below.
   """
    to_report=[]
    for i in range(len(combs.keys())):
        key = list(combs.keys())[i]
        p = calculate_feature_set_probability(df,list(key))
        count = combs[key]
        p_value=calculate_cumulative_binomial_probability(count,p,n)
        if p_value < cutoff:
            #print(key,p_value)
            to_report.append((key,p_value))
    return to_report

In [165]:
to_report = report_pvalues(combs_4,final_df)

In [205]:
#should return True
len(to_report) == 90

True

In [144]:
#need to adjust for the often stupidly large number of tests
#script below does this
def report_pvalues_correct_multi(combs, df, n=11975,alhpa=0.05):
    raw_p_values = []
    for i in range(len(combs.keys())):
        key = list(combs.keys())[i]
        p = calculate_feature_set_probability(df,list(key))
        count = combs[key]
        p_value=calculate_cumulative_binomial_probability(count,p,n)
        raw_p_values.append(p_value)
    adjusted_p_values = multitest.multipletests(raw_p_values)
    return adjusted_p_values

In [218]:
adjusted_4 = report_pvalues_correct_multi(combs_4,final_df)

In [206]:
#[0] is an array of bools. True means reject null. Sun to count significant 
#should return true
sum(adjusted_4[0]) == 19

True

In [207]:
#total number of p-values
#should return true
len(adjusted_4[1]) == 12650

True

In [169]:
def report_adjusted_pvalues(adjusted,combs):
    
    """
    iterates adjusted pvalues returned by report_pvalues_correct_multi
    reports instances where h0 is rejected
    """

    k=0
    n=0
    for i in adjusted[0]:
        
        #if True, then == null rejected
        if i:
            #get the corresponding combination from combs
            combination = list(combs.keys())[n]
            #print index, combination in question, the number of times observed
            print(n,combination,combs[combination])
            #print multiple comparison adjusted p-value
            print(adjusted[1][n])
            print("*****")
            k+=1
        n+=1
    print(k)

In [208]:
report_adjusted_pvalues(adjusted_4, combs)

### Expected output from above:

```
1 ('GntR family transcriptional regulator ', 'MarR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ') 41.0
0.0021472738295903763
*****
10 ('LysR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, MerR family ', 'transcriptional regulator, SARP family ') 24.0
0.002599315574822896
*****
16 ('AraC family transcriptional regulator ', 'LacI family transcription regulator ', 'PadR family transcriptional regulator ', 'TetR family transcriptional regulator ') 21.0
1.404432126149837e-12
*****
21 ('GntR family transcriptional regulator ', 'MarR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'TetR family transcriptional regulator ') 20.0
2.1347069217623614e-08
*****
31 ('LacI family transcription regulator ', 'LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, MarR family ') 17.0
0.00015514900180504406
*****
32 ('LacI family transcription regulator ', 'MarR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'sensor histidine kinase ') 17.0
0.011388223495214888
*****
42 ('MarR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'TetR family transcriptional regulator ') 16.0
0.0013161079207952224
*****
50 ('LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, MarR family ', 'transcriptional regulator, MerR family ') 15.0
0.0045209507813673095
*****
55 ('ArsR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'transcriptional regulator, MerR family ', 'transcriptional regulator, SARP family ') 14.0
1.404432126149837e-12
*****
73 ('AraC family transcriptional regulator ', 'LacI family transcription regulator ', 'PadR family transcriptional regulator ', 'transcriptional regulator, MerR family ') 12.0
1.2636891533111082e-11
*****
79 ('LacI family transcription regulator ', 'LysR family transcriptional regulator ', 'transcriptional regulator, MarR family ', 'transcriptional regulator, MerR family ') 12.0
1.541316562494276e-08
*****
82 ('LacI family transcription regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, MarR family ', 'transcriptional regulator, MerR family ') 12.0
0.0004142525040457837
*****
118 ('AsnC family transcriptional regulator ', 'IclR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'cold-shock DNA-binding domain protein ') 10.0
1.6833816548754517e-09
*****
124 ('LacI family transcription regulator ', 'PadR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, MerR family ') 10.0
0.002638829880397609
*****
152 ('IclR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'autoinducer-binding transcriptional regulator, ', 'biosynthetic-additional (rule-based-clusters) Pkinase') 9.0
5.616840326167819e-12
*****
208 ('ArsR family transcriptional regulator ', 'AsnC family transcriptional regulator ', 'LacI family transcription regulator ', 'transcriptional regulator, SARP family ') 7.0
0.02219051179195983
*****
229 ('GntR family transcriptional regulator ', 'MarR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'transcriptional regulator, MerR family ') 7.0
0.008100109509015012
*****
301 ('LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'biosynthetic-additional (rule-based-clusters) Pkinase', 'sigma-54 dependent trancsriptional regulator ') 6.0
6.577420965948237e-05
*****
439 ('AsnC family transcriptional regulator ', 'LacI family transcription regulator ', 'MarR family transcriptional regulator ', 'sigma-54 dependent trancsriptional regulator ') 4.0
3.537185026059299e-05
*****
19
```

In [176]:
combs_3 = count_feature_combinations(final_df, 3)

In [209]:
#should return True
len(combs_3) == 2300

True

In [178]:
adjusted_3 = report_pvalues_correct_multi(combs_3,final_df)

In [214]:
#should return True
sum(adjusted_3[0]) == 14

True

In [210]:
report_adjusted_pvalues(adjusted_3,combs_3)

### Expected output from above
```
11 ('AraC family transcriptional regulator ', 'GntR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ') 89.0
2.553512956637534e-13
*****
43 ('MarR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'TetR family transcriptional regulator ') 54.0
2.12406739630444e-07
*****
90 ('LacI family transcription regulator ', 'PadR family transcriptional regulator ', 'TetR family transcriptional regulator ') 31.0
0.039820509739227544
*****
93 ('AraC family transcriptional regulator ', 'LacI family transcription regulator ', 'PadR family transcriptional regulator ') 30.0
2.553512956637534e-13
*****
131 ('GntR family transcriptional regulator ', 'MarR family transcriptional regulator ', 'PadR family transcriptional regulator ') 23.0
9.683066730356956e-05
*****
140 ('ArsR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 22.0
1.4999813945055357e-08
*****
212 ('ArsR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'transcriptional regulator, MerR family ') 15.0
5.594881645850163e-08
*****
233 ('IclR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'autoinducer-binding transcriptional regulator, ') 14.0
5.4822542224805363e-08
*****
251 ('LacI family transcription regulator ', 'PadR family transcriptional regulator ', 'transcriptional regulator, MerR family ') 13.0
0.006721714089992982
*****
302 ('AsnC family transcriptional regulator ', 'IclR family transcriptional regulator ', 'cold-shock DNA-binding domain protein ') 10.0
0.005691133234242413
*****
327 ('IclR family transcriptional regulator ', 'autoinducer-binding transcriptional regulator, ', 'biosynthetic-additional (rule-based-clusters) Pkinase') 9.0
6.466960773069795e-06
*****
412 ('LysR family transcriptional regulator ', 'biosynthetic-additional (rule-based-clusters) Pkinase', 'sigma-54 dependent trancsriptional regulator ') 6.0
0.0018603311818455686
*****
451 ('LysR family transcriptional regulator ', 'biosynthetic-additional (rule-based-clusters) Pkinase', 'serine/threonine protein kinase ') 5.0
0.046948230587319735
*****
482 ('AsnC family transcriptional regulator ', 'LacI family transcription regulator ', 'sigma-54 dependent trancsriptional regulator ') 4.0
0.025252912412844396
*****
14
```

In [181]:
combs_5 = count_feature_combinations(final_df, 5)

In [183]:
adjusted_5 = report_pvalues_correct_multi(combs_5,final_df)

In [216]:
#should return True
sum(adjusted_5[0]) == 17

True

In [184]:
report_adjusted_pvalues(adjusted_5,combs_5)

0 ('LysR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'TetR family transcriptional regulator ', 'biosynthetic-additional (rule-based-clusters) Pkinase', 'transcriptional regulator, SARP family ') 16.0
5.742169260442319e-06
*****
2 ('LacI family transcription regulator ', 'LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, MarR family ', 'transcriptional regulator, MerR family ') 11.0
1.3566814337696673e-10
*****
3 ('GntR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'sensor histidine kinase ', 'transcriptional regulator, MerR family ') 10.0
0.00012268228390325818
*****
4 ('AraC family transcriptional regulator ', 'LacI family transcription regulator ', 'PadR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, MerR family ') 9.0
1.179700780810365e-09

### Expected ouptput from above

```
0 ('LysR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'TetR family transcriptional regulator ', 'biosynthetic-additional (rule-based-clusters) Pkinase', 'transcriptional regulator, SARP family ') 16.0
5.742169260442319e-06
*****
2 ('LacI family transcription regulator ', 'LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, MarR family ', 'transcriptional regulator, MerR family ') 11.0
1.3566814337696673e-10
*****
3 ('GntR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'sensor histidine kinase ', 'transcriptional regulator, MerR family ') 10.0
0.00012268228390325818
*****
4 ('AraC family transcriptional regulator ', 'LacI family transcription regulator ', 'PadR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, MerR family ') 9.0
1.179700780810365e-09
*****
10 ('LacI family transcription regulator ', 'LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, MarR family ') 8.0
0.01496879894399153
*****
13 ('GntR family transcriptional regulator ', 'MarR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, MerR family ') 7.0
0.00010038291063565587
*****
19 ('ArsR family transcriptional regulator ', 'AsnC family transcriptional regulator ', 'LacI family transcription regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 6.0
0.008792290928488268
*****
41 ('AraC family transcriptional regulator ', 'LacI family transcription regulator ', 'LysR family transcriptional regulator ', 'cold-shock DNA-binding domain protein ', 'sensor histidine kinase ') 5.0
0.03639911375666295
*****
57 ('GntR family transcriptional regulator ', 'MarR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, MerR family ') 5.0
0.003911442950018904
*****
65 ('LacI family transcription regulator ', 'LysR family transcriptional regulator ', 'MarR family transcriptional regulator ', 'cold-shock DNA-binding domain protein ', 'sensor histidine kinase ') 5.0
0.04225908457734244
*****
99 ('ArsR family transcriptional regulator ', 'GntR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ') 4.0
0.016693092085061045
*****
101 ('ArsR family transcriptional regulator ', 'GntR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 4.0
0.04188279604890475
*****
104 ('ArsR family transcriptional regulator ', 'GntR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'transcriptional regulator, SARP family ') 4.0
0.028224770429318244
*****
137 ('GntR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'autoinducer-binding transcriptional regulator, ', 'sensor histidine kinase ', 'transcriptional regulator, SARP family ') 4.0
0.0004733034859256006
*****
152 ('AraC family transcriptional regulator ', 'ArsR family transcriptional regulator ', 'AsnC family transcriptional regulator ', 'LacI family transcription regulator ', 'PadR family transcriptional regulator ') 3.0
0.010440301384798144
*****
196 ('ArsR family transcriptional regulator ', 'AsnC family transcriptional regulator ', 'LacI family transcription regulator ', 'PadR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 3.0
0.043231388074873846
*****
262 ('LysR family transcriptional regulator ', 'cold-shock DNA-binding domain protein ', 'sensor histidine kinase ', 'sigma-54 dependent trancsriptional regulator ', 'transcriptional regulator, SARP family ') 3.0
0.0022445543650219083
*****
17
```

In [185]:
combs_2 = count_feature_combinations(final_df, 2)

In [219]:
#should return True
len(combs_2) == 300

True

In [186]:
adjusted_2 = report_pvalues_correct_multi(combs_2,final_df)

In [220]:
#should return True
sum(adjusted_2[0]) == 2

True

In [221]:
report_adjusted_pvalues(adjusted_2,combs_2)

### Expected output from above

```
61 ('MarR family transcriptional regulator ', 'PadR family transcriptional regulator ') 81.0
0.0007064134431359788
*****
154 ('biosynthetic-additional (rule-based-clusters) Pkinase', 'serine/threonine protein kinase ') 9.0
0.04612172429346162
*****
2
```

In [187]:
#this one will take a while to run
combs_6 = count_feature_combinations(final_df, 6)

In [224]:
len(combs_6) == 177100

True

In [188]:
#this one will take a while to run
adjusted_6 = report_pvalues_correct_multi(combs_6,final_df)

In [229]:
#should return True
sum(adjusted_6[0]) == 14

True

In [230]:
report_adjusted_pvalues(adjusted_6,combs_6)

### Expected output from above

```
0 ('GntR family transcriptional regulator ', 'MarR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, MerR family ') 5.0
0.00017982861372431688
*****
1 ('LacI family transcription regulator ', 'LysR family transcriptional regulator ', 'MarR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'cold-shock DNA-binding domain protein ', 'sensor histidine kinase ') 5.0
0.0020403196141226647
*****
4 ('ArsR family transcriptional regulator ', 'GntR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'TetR family transcriptional regulator ') 4.0
0.0017940468854606673
*****
5 ('ArsR family transcriptional regulator ', 'GntR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'transcriptional regulator, SARP family ') 4.0
8.011930883691676e-05
*****
6 ('ArsR family transcriptional regulator ', 'GntR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 4.0
0.004583360984948673
*****
8 ('ArsR family transcriptional regulator ', 'GntR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 4.0
0.0030601064830237234
*****
10 ('ArsR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 4.0
0.010522835344247748
*****
14 ('LacI family transcription regulator ', 'LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, MarR family ', 'transcriptional regulator, MerR family ') 4.0
0.043665819921042584
*****
15 ('AraC family transcriptional regulator ', 'ArsR family transcriptional regulator ', 'AsnC family transcriptional regulator ', 'LacI family transcription regulator ', 'PadR family transcriptional regulator ', 'TetR family transcriptional regulator ') 3.0
0.0026071569079719097
*****
16 ('AraC family transcriptional regulator ', 'ArsR family transcriptional regulator ', 'AsnC family transcriptional regulator ', 'LacI family transcription regulator ', 'PadR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 3.0
0.00025204035521723245
*****
18 ('AraC family transcriptional regulator ', 'ArsR family transcriptional regulator ', 'AsnC family transcriptional regulator ', 'PadR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 3.0
0.03228691108523353
*****
29 ('ArsR family transcriptional regulator ', 'AsnC family transcriptional regulator ', 'LacI family transcription regulator ', 'PadR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 3.0
0.010964606182162983
*****
48 ('AraC family transcriptional regulator ', 'AsnC family transcriptional regulator ', 'LacI family transcription regulator ', 'LysR family transcriptional regulator ', 'MarR family transcriptional regulator ', 'sigma-54 dependent trancsriptional regulator ') 2.0
0.009264475031112687
*****
111 ('AsnC family transcriptional regulator ', 'GntR family transcriptional regulator ', 'LacI family transcription regulator ', 'MarR family transcriptional regulator ', 'sigma-54 dependent trancsriptional regulator ', 'transcriptional regulator, SARP family ') 2.0
0.012973427227664319
*****
14
```

In [233]:
def report_n_highest(combinations, n=10):
    """
    reports the n most frequently occuring combinations, regardless of p-value
    """
    for i in range(n):
        key = list(combinations.keys())[i]
        count = combinations[key]
        print(key, count)
        print("***")
    

In [239]:
report_n_highest(combs_2)

### Expected output from above
```
('TetR family transcriptional regulator ', 'sensor histidine kinase ') 737.0
***
('TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 718.0
***
('LysR family transcriptional regulator ', 'TetR family transcriptional regulator ') 676.0
***
('MarR family transcriptional regulator ', 'TetR family transcriptional regulator ') 530.0
***
('sensor histidine kinase ', 'transcriptional regulator, SARP family ') 481.0
***
('RNA polymerase, sigma-24 subunit, ECF subfamily ', 'TetR family transcriptional regulator ') 433.0
***
('GntR family transcriptional regulator ', 'TetR family transcriptional regulator ') 426.0
***
('AraC family transcriptional regulator ', 'TetR family transcriptional regulator ') 387.0
***
('TetR family transcriptional regulator ', 'transcriptional regulator, MerR family ') 342.0
***
('LysR family transcriptional regulator ', 'sensor histidine kinase ') 323.0
***
```

In [240]:
report_n_highest(combs_3)

### Expected output from above
```
('TetR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, SARP family ') 197.0
***
('LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ') 176.0
***
('MarR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ') 149.0
***
('LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 145.0
***
('GntR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ') 121.0
***
('GntR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 107.0
***
('LysR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'TetR family transcriptional regulator ') 103.0
***
('RNA polymerase, sigma-24 subunit, ECF subfamily ', 'TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 98.0
***
('LysR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, SARP family ') 97.0
***
('MarR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 92.0
***
```

In [241]:
report_n_highest(combs_4)

### Expected output from above
```
('LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, SARP family ') 52.0
***
('GntR family transcriptional regulator ', 'MarR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ') 41.0
***
('GntR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, SARP family ') 32.0
***
('LacI family transcription regulator ', 'LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ') 32.0
***
('MarR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, SARP family ') 32.0
***
('TetR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, MerR family ', 'transcriptional regulator, SARP family ') 32.0
***
('LysR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 31.0
***
('GntR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 28.0
***
('LysR family transcriptional regulator ', 'MarR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ') 28.0
***
('LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, MerR family ') 27.0
***
```

In [242]:
report_n_highest(combs_5)

### Expected output from above
```
('LysR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'TetR family transcriptional regulator ', 'biosynthetic-additional (rule-based-clusters) Pkinase', 'transcriptional regulator, SARP family ') 16.0
***
('LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, MerR family ', 'transcriptional regulator, SARP family ') 14.0
***
('LacI family transcription regulator ', 'LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, MarR family ', 'transcriptional regulator, MerR family ') 11.0
***
('GntR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'sensor histidine kinase ', 'transcriptional regulator, MerR family ') 10.0
***
('AraC family transcriptional regulator ', 'LacI family transcription regulator ', 'PadR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, MerR family ') 9.0
***
('GntR family transcriptional regulator ', 'MarR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, MerR family ') 9.0
***
('LysR family transcriptional regulator ', 'MarR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, SARP family ') 9.0
***
('GntR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 8.0
***
('GntR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, SARP family ') 8.0
***
('LacI family transcription regulator ', 'LysR family transcriptional regulator ', 'MarR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ') 8.0
***
```

In [243]:
report_n_highest(combs_6)

### Expected output from above

```
('GntR family transcriptional regulator ', 'MarR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, MerR family ') 5.0
***
('LacI family transcription regulator ', 'LysR family transcriptional regulator ', 'MarR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'cold-shock DNA-binding domain protein ', 'sensor histidine kinase ') 5.0
***
('AraC family transcriptional regulator ', 'GntR family transcriptional regulator ', 'IclR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 4.0
***
('AraC family transcriptional regulator ', 'LacI family transcription regulator ', 'LysR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'cold-shock DNA-binding domain protein ', 'sensor histidine kinase ') 4.0
***
('ArsR family transcriptional regulator ', 'GntR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'TetR family transcriptional regulator ') 4.0
***
('ArsR family transcriptional regulator ', 'GntR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'transcriptional regulator, SARP family ') 4.0
***
('ArsR family transcriptional regulator ', 'GntR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 4.0
***
('ArsR family transcriptional regulator ', 'GntR family transcriptional regulator ', 'LysR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 4.0
***
('ArsR family transcriptional regulator ', 'GntR family transcriptional regulator ', 'PadR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'TetR family transcriptional regulator ', 'transcriptional regulator, SARP family ') 4.0
***
('ArsR family transcriptional regulator ', 'GntR family transcriptional regulator ', 'RNA polymerase, sigma-24 subunit, ECF subfamily ', 'TetR family transcriptional regulator ', 'sensor histidine kinase ', 'transcriptional regulator, MerR family ') 4.0
***
```